In [1]:
import h5py
import glob
import os


#dataset_root = '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200/'
#files = sorted( filter( os.path.isdir,
#                        glob.glob(dataset_root + '*') ) )
#for file in files:
#    print(file)

In [2]:
import pyvista as pv
def plotMesh(meshfile, arr, property):
    p = pv.Plotter()
    p.enable()
    p.enable_anti_aliasing()

    mesh = pv.read(meshfile)
    mesh[property] = arr[:]
    mesh.set_active_scalars(property)
    p.add_mesh(mesh, opacity=0.85, render=True, cmap='plasma')
    p.add_mesh(mesh.contour(), color="white", line_width=2, render=True, cmap='plasma')

    p.set_viewup([0, 1, 0])
    #p.fly_to([5,0,0])

    #p.set_position([5.0, -0.01, 7.5])
    p.window_size = [1280,480]
    #p.save_graphic('./annSU2_'+str(pod.modes.shape[1])+'_modes.pdf')
    p.show(interactive_update=False, auto_close=True)
    #p.show()

In [7]:
import h5py
import numpy as np
import glob
import pyvista as pv
import pandas as pd

def genQ1DHDF5(path, variables, outputfile, solutions_id):
    #path = path_ + 'Q1D/*'
    print(f'Reading data from {path}, generating HDF5 file...')
    #npaths = len(list(filter( os.path.isdir, glob.glob(f'{path}*') ))) 

    arrsize = len(np.loadtxt(f"{path}/{1}/Q1D/outputs/{variables[0]}"))
    arrsize = arrsize*len(variables)

    snaps = h5py.File(outputfile,'w')
    snaps.create_dataset('snapshots', shape=(arrsize, len(solutions_id)), dtype=np.float64)
    #snaps.create_dataset('id', shape=(len(solutions_id)), dtype=np.int32)
    #snaps.create_dataset('p0in', shape=(len(solutions_id)), dtype=np.float64)
    #snaps.create_dataset('T0in', shape=(len(solutions_id)), dtype=np.float64)
    #snaps.create_dataset('thickness', shape=(len(solutions_id)), dtype=np.float64)
    
    df = pd.read_csv(f"{path}/doe_lhs.txt")
    doe_params = list(df.keys())

    for param in doe_params: 
        snaps.create_dataset(param, shape=(len(solutions_id)), dtype=type(df[param][0]))
   

    #for i in range(1, npaths+1):
    for i,j in enumerate(solutions_id):
        idx0 = 0
        idxf = idx0
        for var in variables:
            #filename = path.split('*')[0]+str(i)+'/outputs/'+var
            filename = f"{path}/{j}/Q1D/outputs/{var}"
            print(f"reading file {filename}")
            arr = np.loadtxt(filename)
            idx0 = idxf
            idxf = idx0 + len(arr)
            snaps['snapshots'][idx0:idxf,i-1] = arr[:]
    
        #snaps['DoE'] = df

        #snaps['id'][i] = j

        doe_params = get_doe_parameters(f"{path}/doe_lhs.txt", j)
       
        for key, val in doe_params.items():
            snaps[key][i] = val

        #p0in, T0in, thickness = get_boundary_condition(f"{path}/doe_lhs.txt", j)
        #snaps['p0in'][i] = p0in
        #snaps['T0in'][i] = T0in
        #snaps['thickness'][i] = thickness

    snaps.close()

    doe = pd.read_csv(f"{path}/doe_lhs.txt")
    doe.to_hdf(outputfile, key='DoE', mode='a', append=True)    

    print(f'File {outputfile} written')

def get_wall_idx(vtkfile):
    output = pv.read(vtkfile)
    #edges = output.extract_feature_edges()
    edges = output
    wall_idx = np.where(edges['Heat_Flux'] != 0)[0]
    return wall_idx

def genSU2HDF5_Fluid(path, vtkfile, variables, outputfile, solutions_id):
    #path = path_ + 'SU2/*'
    print(f'Reading data from {path}, generating HDF5 file...')
    #npaths = len(list(filter( os.path.isdir, glob.glob(f'{path}*') )))

    #mesh = pv.read(path.split('*')[0]+'1/outputs/solution.vtk')
    mesh = pv.read(f"{path}/1/SU2/outputs/{vtkfile}")
    if 'Heat_Flux' in variables:
        wall_idx = get_wall_idx(f"{path}/1/SU2/outputs/{vtkfile}")
        arrsize = len(mesh[variables[0]])
        arrsize = arrsize*(len(variables)-1) + len(wall_idx)
    else:
        arrsize = len(mesh[variables[0]])
        arrsize = arrsize*len(variables)

    snaps = h5py.File(outputfile,'w')
    snaps.create_dataset('snapshots', shape=(arrsize, len(solutions_id)), dtype=np.float64)
    #snaps.create_dataset('id', shape=(len(solutions_id)), dtype=np.int32)
    #snaps.create_dataset('p0in', shape=(len(solutions_id)), dtype=np.float64)
    #snaps.create_dataset('T0in', shape=(len(solutions_id)), dtype=np.float64)
    #snaps.create_dataset('thickness', shape=(len(solutions_id)), dtype=np.float64)

    df = pd.read_csv(f"{path}/doe_lhs.txt")
    doe_params = list(df.keys())

    for param in doe_params: 
        snaps.create_dataset(param, shape=(len(solutions_id)), dtype=type(df[param][0]))
    
    meshfiles=[]
    for i,j in enumerate(solutions_id):
        idx0 = 0
        idxf = idx0
        #filename = path.split('*')[0]+str(i)+'/outputs/solution.vtk'
        filename = f"{path}/{j}/SU2/outputs/{vtkfile}"
        meshfile = f"{path}/{j}/SU2/outputs/{vtkfile.split('.vtk')[0]}_mesh.vtk"
        print(f'reading file {filename}')
        mesh = pv.read(filename)
        for var in variables:
            if var == 'Heat_Flux':
                arr = mesh[var][wall_idx]
                
                #import matplotlib.pyplot as plt
                #print(var)
                #print(mesh[var].shape)
                #plotMesh(f"{path}/{j}/SU2/outputs/{vtkfile}", mesh[var], var)
                #plt.plot(mesh[var][wall_idx])
                #plt.show()
                #input()
                
                idx0 = idxf
                idxf = idx0 + len(arr)
                snaps['snapshots'][idx0:idxf,i-1] = arr[:]
            else:
                arr = mesh[var]
                idx0 = idxf
                idxf = idx0 + len(arr)
                snaps['snapshots'][idx0:idxf,i-1] = arr[:]
        meshfiles.append(meshfile)

        #snaps['id'][i] = j

        doe_params = get_doe_parameters(f"{path}/doe_lhs.txt", j)
       
        for key, val in doe_params.items():
            snaps[key][i] = val

        #p0in, T0in, thickness = get_boundary_condition(f"{path}/doe_lhs.txt", j)
        #snaps['p0in'][i] = p0in
        #snaps['T0in'][i] = T0in
        #snaps['thickness'][i] = thickness

         

    snaps.create_dataset("meshfile", data=np.array(meshfiles, dtype='S'))
    snaps.close()

    doe = pd.read_csv(f"{path}/doe_lhs.txt")
    doe.to_hdf(outputfile, key='DoE', mode='a', append=True)

    print(f'File {outputfile} written')

def genSU2HDF5(path, vtkfile, variables, outputfile, solutions_id):
    #path = path_ + 'SU2/*'
    print(f'Reading data from {path}, generating HDF5 file...')
    #npaths = len(list(filter( os.path.isdir, glob.glob(f'{path}*') )))

    #mesh = pv.read(path.split('*')[0]+'1/outputs/solution.vtk')
    mesh = pv.read(f"{path}/1/SU2/outputs/{vtkfile}")
    arrsize = len(mesh[variables[0]])
    arrsize = arrsize*len(variables)

    snaps = h5py.File(outputfile,'w')
    snaps.create_dataset('snapshots', shape=(arrsize, len(solutions_id)), dtype=np.float64)
    snaps.create_dataset('id', shape=(len(solutions_id)), dtype=np.int32)

    df = pd.read_csv(f"{path}/doe_lhs.txt")
    doe_params = list(df.keys())

    for param in doe_params: 
        snaps.create_dataset(param, shape=(len(solutions_id)), dtype=type(df[param][0]))
    
        
    meshfiles=[]
    for i,j in enumerate(solutions_id):
        idx0 = 0
        idxf = idx0
        #filename = path.split('*')[0]+str(i)+'/outputs/solution.vtk'
        filename = f"{path}/{j}/SU2/outputs/{vtkfile}"
        meshfile = f"{path}/{j}/SU2/outputs/{vtkfile.split('.vtk')[0]}_mesh.vtk"
        print(f'reading file {filename}')
        mesh = pv.read(filename)
        for var in variables:
            arr = mesh[var]
            idx0 = idxf
            idxf = idx0 + len(arr)
            snaps['snapshots'][idx0:idxf,i-1] = arr[:]
        meshfiles.append(meshfile)

        snaps['id'][i] = j

        #p0in, T0in, thickness = get_boundary_condition(f"{path}/doe_lhs.txt", j)
        doe_params = get_doe_parameters(f"{path}/doe_lhs.txt", j)
       
        for key, val in doe_params.items():
            snaps[key][i] = val
        
        #snaps['p0in'][i] = p0in
        #snaps['T0in'][i] = T0in
        #snaps['thickness'][i] = thickness

    snaps.create_dataset("meshfile", data=np.array(meshfiles, dtype='S'))
    snaps.close()

    doe = pd.read_csv(f"{path}/doe_lhs.txt")
    doe.to_hdf(outputfile, key='DoE', mode='a', append=True)

    print(f'File {outputfile} written')


def _get_converged_solutions(path):
    # old implementations, dooest not take the solution filename for verification
    solutions_id = []
    n_directories = len(list(filter( os.path.isdir, glob.glob(f'{path}/*') ))) 
    for i in range(n_directories):
        solution_file = f'{path}/{i+1}/SU2/outputs/fluid.vtk'
        if len(list( glob.glob(f'{solution_file}*') )) > 0:
            solutions_id.append(i+1)

    return solutions_id

def get_converged_solutions(path, after_path):
    solutions_id = []
    n_directories = len(list(filter( os.path.isdir, glob.glob(f'{path}/*') ))) 
    for i in range(n_directories):
        solution_file = f'{path}/{i+1}{after_path}'
        if len(list( glob.glob(f'{solution_file}*') )) > 0:
            solutions_id.append(i+1)

    return solutions_id

def get_boundary_condition(doe_file, id):
    #dataset_root = '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200'
    #id = 6 # case to run
    #doe_file = '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200/doe_lhs.txt'
    doe_table = np.loadtxt(doe_file, skiprows=1, delimiter=',')
    idx = np.where(doe_table[:,0] == id) # idx related to case number

    p0in = doe_table[idx,1][0][0]
    T0in = doe_table[idx,2][0][0]
    thickness = doe_table[idx,3][0][0]

    return p0in, T0in, thickness

def get_doe_parameters(doe_file, id):
    df = pd.read_csv(doe_file, delimiter=',')

    d = {}
    for key in df.keys():
        d[key] = df[df['ID']==id][key][0]
    return d
    #return tuple(d.values())

def main():
    
    solutions_id = _get_converged_solutions('/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200')

    genQ1DHDF5('/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200', 
               ['p.txt','T.txt','M.txt'],
               '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200/Q1D.hdf5', 
               solutions_id)       
    
    genSU2HDF5_Fluid('/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200', 
               'fluid.vtk',
               ['Pressure','Temperature','Mach','Heat_Flux'],
               '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200/SU2_fluid.hdf5',
               solutions_id)
    
    #genSU2HDF5('/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200', 
    #           'solid.vtk',
    #           ['Temperature'],
    #           '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200/SU2_solid.hdf5',
    #           solutions_id)
    

In [8]:
doe_path = '/home/ppiper/ihtc_repository/data/doe_test'
su2_after_path = '/SU2/outputs/cht_setupSU2.vtm'

q1d_variables =  ['p.txt','T.txt','M.txt']
d1d_hdf5_file =  f'{doe_path}Q1D.hdf5'


In [9]:
solutions_id = get_converged_solutions(path=doe_path, after_path=su2_after_path)
solutions_id

[1]

In [10]:
genQ1DHDF5( path = doe_path, variables=q1d_variables, outputfile=d1d_hdf5_file, 
            solutions_id = solutions_id)  

Reading data from /home/ppiper/ihtc_repository/data/doe_test, generating HDF5 file...
reading file /home/ppiper/ihtc_repository/data/doe_test/1/Q1D/outputs/p.txt
reading file /home/ppiper/ihtc_repository/data/doe_test/1/Q1D/outputs/T.txt
reading file /home/ppiper/ihtc_repository/data/doe_test/1/Q1D/outputs/M.txt


ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [ ]:

     

genSU2HDF5_Fluid('/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200', 
            'fluid.vtk',
            ['Pressure','Temperature','Mach','Heat_Flux'],
            '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_N200/SU2_fluid.hdf5',
            solutions_id)

In [ ]:
import pyvista as pv
multiblock = pv.read('/home/ppiper/ihtc_repository/data/doe_test/1/SU2/outputs/cht_setupSU2.vtm')

In [ ]:
multiblock.keys()

In [ ]:
multiblock['Zone 0 (Comp. Fluid)']['Internal']['Internal']

In [ ]:
points[:,0]

In [ ]:

multiblock['Zone 1 (Solid Heat)']['Boundary']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
points = multiblock['Zone 1 (Solid Heat)']['Boundary']['INNERWALL'].points
data =  multiblock['Zone 1 (Solid Heat)']['Boundary']['INNERWALL']['Temperature']

idx_sort = np.argsort(points[:,0])
points[:,0] = points[idx_sort,0]
points[:,1] = points[idx_sort,1]
data = data[idx_sort]

plt.plot(points[:,0], data)
#plt.plot(points[:,0], points[:,1])

In [ ]:
pl = pv.Plotter()
mesh_fluid = multiblock['Zone 0 (Comp. Fluid)']['Internal']['Internal']
mesh_fluid.set_active_scalars('Pressure')
mesh_solid = multiblock['Zone 1 (Solid Heat)']['Internal']['Internal']
mesh_solid.set_active_scalars('Temperature')
pl.add_mesh(mesh_fluid)
pl.add_mesh(mesh_solid)
pl.show(cpos='xy')

In [ ]:
multiblock['Zone 0 (Comp. Fluid)']['Internal']['Internal']['Pressure']

In [ ]:
multiblock.GetBlock(0)[1][1]['Density']

In [ ]:
multiblock.GetBlock(0).set_active_scalars('Pressure')

In [ ]:
multiblock.GetBlock(0).plot()

In [ ]:
multiblock[0][0][0].set_active_scalars('Pressure')

In [ ]:
from pyvista import GPUInfo
GPUInfo()

In [ ]:
multiblock.plot()

In [ ]:
dir(multiblock[0][0][0])

In [ ]:
multiblock.set_active_scalars('PRESSURE')

In [ ]:
multiblock.plot()